In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
import urllib.request

url = "https://github.com/eishkina-estia/ML2023/raw/main/data/New_York_City_Taxi_Trip_Duration.zip"
output_file = "New_York_City_Taxi_Trip_Duration.zip"

urllib.request.urlretrieve(url, output_file)
print("Fichier téléchargé avec succès !")


Fichier téléchargé avec succès !


In [51]:
import os

path = 'New_York_City_Taxi_Trip_Duration.zip'
data = pd.read_csv(path, compression='zip')
os.remove(path)

In [52]:
import sys
sys.path.append('../')
import common

In [53]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['id', 'trip_duration', 'dropoff_datetime'])
# Convertir 'pickup_datetime' en datetime
X['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])

# Extraire des informations temporelles et les convertir en entiers
X['pickup_hour'] = X['pickup_datetime'].dt.hour               # Heure de la journée (de 0 à 23)
X['pickup_dayofweek'] = X['pickup_datetime'].dt.dayofweek     # Jour de la semaine (0=Monday, 6=Sunday)
X['pickup_month'] = X['pickup_datetime'].dt.month             # Mois (1 à 12)
X['pickup_year'] = X['pickup_datetime'].dt.year               # Année (par exemple, 2021)
X['pickup_day'] = X['pickup_datetime'].dt.day                 # Jour du mois (1 à 31)

# Supprimer la colonne 'pickup_datetime' maintenant qu'on a extrait les informations
X = X.drop(columns=['pickup_datetime'])

#Remplacement des variables catégorielles
X['vendor_id'] = X['vendor_id'].map({1: 0, 2: 1})
X['store_and_fwd_flag'] = X['store_and_fwd_flag'].map({'N': 0, 'Y': 1})

y = data['trip_duration']
display(X.shape)
X.head()

(1458644, 12)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_hour,pickup_dayofweek,pickup_month,pickup_year,pickup_day
0,1,1,-73.982155,40.767937,-73.964630,40.765602,0,17,0,3,2016,14
1,0,1,-73.980415,40.738564,-73.999481,40.731152,0,0,6,6,2016,12
2,1,1,-73.979027,40.763939,-74.005333,40.710087,0,11,1,1,2016,19
3,1,1,-74.010040,40.719971,-74.012268,40.706718,0,19,2,4,2016,6
4,1,1,-73.973053,40.793209,-73.972923,40.782520,0,13,5,3,2016,26


In [54]:
X.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_hour,pickup_dayofweek,pickup_month,pickup_year,pickup_day
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1458644.0,1.458644e+06
mean,5.349503e-01,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,5.515396e-03,1.360648e+01,3.050375e+00,3.516818e+00,2016.0,1.550402e+01
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,7.406066e-02,6.399693e+00,1.954039e+00,1.681038e+00,0.0,8.703135e+00
min,0.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2016.0,1.000000e+00
25%,0.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,0.000000e+00,9.000000e+00,1.000000e+00,2.000000e+00,2016.0,8.000000e+00
50%,1.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,0.000000e+00,1.400000e+01,3.000000e+00,4.000000e+00,2016.0,1.500000e+01
75%,1.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,0.000000e+00,1.900000e+01,5.000000e+00,5.000000e+00,2016.0,2.300000e+01
max,1.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,1.000000e+00,2.300000e+01,6.000000e+00,6.000000e+00,2016.0,3.100000e+01


In [55]:
#sns.pairplot(X);

In [56]:
# Plot the target distribution
#sns.displot(y, kde=True, height=4, aspect=2)
#plt.title("Target distribution");

In [57]:
X.isna().any()

vendor_id             False
passenger_count       False
pickup_longitude      False
pickup_latitude       False
dropoff_longitude     False
dropoff_latitude      False
store_and_fwd_flag    False
pickup_hour           False
pickup_dayofweek      False
pickup_month          False
pickup_year           False
pickup_day            False
dtype: bool

In [58]:
from sklearn.model_selection import train_test_split
data = pd.concat([X,y], axis=1)
data_train, data_test = train_test_split(data, test_size=0.2, random_state=common.RANDOM_STATE)
display(data_train.shape, data_test.shape)

(1166915, 13)

(291729, 13)

In [59]:
import sqlite3
import os

db_dir = os.path.dirname(common.DB_PATH)
if not os.path.exists(db_dir):
    os.makedirs(db_dir)

print(f"Saving train and test data to a database: {common.DB_PATH}")
with sqlite3.connect(common.DB_PATH) as con:
    # cur = con.cursor()
    # cur.execute("DROP TABLE IF EXISTS train")
    # cur.execute("DROP TABLE IF EXISTS test")
    data_train.to_sql(name='train', con=con, if_exists="replace")
    data_test.to_sql(name='test', con=con, if_exists="replace")

Saving train and test data to a database: C:\ESTIA\MLops\MyProject\data\NYC_taxi.db


In [60]:
print(f"Reading train data from the database: {common.DB_PATH}")
with sqlite3.connect(common.DB_PATH) as con:
    cur = con.cursor()
    res = cur.execute("SELECT * FROM train LIMIT 3")
    display(res.fetchall())

Reading train data from the database: C:\ESTIA\MLops\MyProject\data\NYC_taxi.db


[(1053743,
  1,
  1,
  -73.86276245117188,
  40.768821716308594,
  -73.89170074462889,
  40.74668884277344,
  0,
  18,
  2,
  6,
  2016,
  29,
  1133),
 (273748,
  1,
  1,
  -73.95803833007811,
  40.78323745727539,
  -73.97550964355469,
  40.7608528137207,
  0,
  13,
  0,
  4,
  2016,
  25,
  887),
 (433988,
  1,
  1,
  -73.96945953369139,
  40.78551864624024,
  -73.98924255371094,
  40.77174758911133,
  0,
  12,
  5,
  5,
  2016,
  7,
  686)]